---
title: Importing org units to DHIS2
short_title: Import to DHIS2
---

Here we show how to generate and import an entirely new DHIS2 org unit hierarchy programmatically from local org unit boundary data. 

In [2]:
import geopandas as gpd
from dhis2eo.integrations.geopandas import geodataframe_to_dhis2_org_units

## Load your local boundary data

First, we load your local boundary data into a GeoPandas dataframe. This is typically only for a single organisation unit level, e.g. provinces. The current functionality currently only supports the first topmost level under the national. 

Using `geopandas` we can load the data from any file format, such as shapefiles:

In [ ]:
gdf = gpd.read_file('../data/sierra-leone-districts.shp')

Or GeoJSON files:

In [18]:
gdf = gpd.read_file('../data/sierra-leone-districts.geojson')

Let's inspect the contents of the organisation units data:

In [17]:
gdf

,type,id,name,hasCoordinatesDown,hasCoordinatesUp,level,grandParentParentGraph,grandParentId,parentGraph,parentId,parentName,dimensions,weight,geometry
0,Polygon,O6uvpzGd5pu,Bo,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,Polygon,fdc6uOvgoji,Bombali,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,MultiPolygon,lc3eMKXaEfw,Bonthe,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,Polygon,jUb8gELQApl,Kailahun,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,MultiPolygon,PMa2VCrupOd,Kambia,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,Polygon,kJq2mPyFEHo,Kenema,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,Polygon,qhqAxPSTUXp,Koinadugu,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,Polygon,Vth0fbpFcsO,Kono,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,MultiPolygon,jmIPBj66vD6,Moyamba,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,MultiPolygon,TEQlaapDQoK,Port Loko,True,False,2,,,ImspTQPwCqd,ImspTQPwCqd,Sierra Leone,{ },1,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


## Convert to DHIS2 format

Next, we simply pass this geodataframe to our `dhis2eo` utility function to convert to the formats required by DHIS2. What this does is create organisation units for each of the entries in the geodataframe, as well as a toplevel organisation unit for the entire country. More complex organisation unit hierarchies with multiple nested levels are not currently supported. 

In [ ]:
dhis2_metadata, dhis2_geojson = geodataframe_to_dhis2_org_units(gdf, country='Sierra Leone', name_field='Name')

This function returns two results. First, the DHIS2 JSON payload to generate the org unit metadata: 

In [9]:
dhis2_metadata['organisationUnits'][:3]

[{'id': 'NpciiKIqKaU',
  'name': 'Sierra Leone',
  'shortName': 'Sierra Leone',
  'openingDate': '2025-10-03',
  'level': 1},
 {'id': 'R3bFNJaZAT8',
  'name': None,
  'shortName': 'Unnamed',
  'openingDate': '2025-10-03',
  'level': 2,
  'parent': {'id': 'NpciiKIqKaU'}},
 {'id': 'P9yMVrCLXNs',
  'name': None,
  'shortName': 'Unnamed',
  'openingDate': '2025-10-03',
  'level': 2,
  'parent': {'id': 'NpciiKIqKaU'}}]

Second, the function returns a GeoJSON containing the geometries that match and link with the org unit metadata:  

In [14]:
dhis2_geojson['features'][0] # showing first org unit only

{'id': 'R3bFNJaZAT8',
 'type': 'Feature',
 'properties': {'id': 'R3bFNJaZAT8',
  'name': None,
  'shortName': 'Unnamed',
  'openingDate': '2025-10-03',
  'level': 2,
  'parent': {'id': 'NpciiKIqKaU'}},
 'geometry': {'type': 'Polygon',
  'coordinates': (((-11.5914, 8.4875),
    (-11.5906, 8.4769),
    (-11.5898, 8.4717),
    (-11.5882, 8.466),
    (-11.5863, 8.4604),
    (-11.5837, 8.4549),
    (-11.5804, 8.4505),
    (-11.5772, 8.4477),
    (-11.5738, 8.4456),
    (-11.5622, 8.44),
    (-11.558, 8.4365),
    (-11.5567, 8.4332),
    (-11.5565, 8.4295),
    (-11.558, 8.4215),
    (-11.5582, 8.4185),
    (-11.5569, 8.4121),
    (-11.5567, 8.4084),
    (-11.5571, 8.4047),
    (-11.5589, 8.3968),
    (-11.5588, 8.3927),
    (-11.5574, 8.3877),
    (-11.5549, 8.3835),
    (-11.5506, 8.3751),
    (-11.5488, 8.3707),
    (-11.5438, 8.3562),
    (-11.5418, 8.3519),
    (-11.5398, 8.3489),
    (-11.5361, 8.3451),
    (-11.5278, 8.3419),
    (-11.516, 8.3381),
    (-11.5114, 8.3375),
    (-11.494

## Import to DHIS2

Next, we use the `python-dhis2-client` to push these results to our DHIS2 instance. 

We start by connecting to our DHIS2 instance:

In [15]:
from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

# Client configuration
cfg = ClientSettings(
  base_url="http://localhost:8080",
  username="admin",
  password="district")

client = DHIS2Client(settings=cfg)
info = client.get_system_info()

# Check if everything is working.
# You should see your current DHIS2 version info.
print("▶ Current DHIS2 version:", info["version"])

ConnectError: [WinError 10061] No connection could be made because the target machine actively refused it

Then, we can use the `dhis2-python-client` to push the metadata and create the organisation unit hierarchy to DHIS2: 

Lastly, we use the client to push the geojson file to import and attach geometries to each of the DHIS2 organisation units: 

To confirm that DHIS2 now contains the imported organisation units with boundary geometries, we can use the client to get the current organisation units:

In [ ]:
for ou in islice(client.get_organisation_units(level=3, fields="id,name", order="name:asc"), 10):
    print(f"{ou['id']} · {ou['name']}")